# dim_geolocalizacao

Esse notebook é responsável por criar uma tabela com as informações básicas das localizações.


## Configurações Iniciais

### Preparação do Ambiente PySpark

#### Instalando as Dependências

In [1]:
# Instala o Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixa o Apache Spark 3.4.1 com Hadoop 3 (versão estável e recente)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompacta o arquivo baixado
!tar xf spark-3.4.1-bin-hadoop3.tgz

# Instala a biblioteca findspark
!pip install -q findspark

#### Editando as Variáveis de Ambiente

In [2]:
import os

# Define a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Define a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

from pyspark.sql import SparkSession

# Cria a SparkSession
spark = SparkSession.builder.master("local[*]").appName("E-commerce Dataset").getOrCreate()

### Importando as Bibliotecas

In [3]:
from pyspark.sql import functions as F

### Carregando os Dados

In [14]:
df_geolocation_dataset = (
    spark.read.csv(
        "/content/olist_geolocation_dataset.csv",
        header=True,
        inferSchema=True
    )
)

## Transformação dos Dados

In [ ]:
df_final_geolocation_dataset = (
    df_geolocation_dataset
    .withColumn(
        "geolocation_country",
        F.lit("BR")
    )
    .withColumn(
        "geolocation_city",
        F.upper(F.col("geolocation_city"))
    )
    .withColumn(
        "geolocation_city_state",
        F.concat_ws(
            " - ",
            F.col("geolocation_city"),
            F.col("geolocation_state")
        )
    )
)

## Salvamento dos Dados

In [ ]:
(
    df_final_geolocation_dataset
    .coalesce(1)
    .write
    .option("header", "true")
    .mode("overwrite")
    .csv("/transformed_data/dim_geolocalizacao")
)